## Non Compliance Experiment=5
Test top norms for different w_nc, and relation between violations with diff params   
Dask page: http://localhost:8787/info/main/workers.html

In [1]:
import sys
sys.path.append('../src')

In [2]:
import yaml
from IPython.utils import io
from tqdm.notebook import tqdm
from pathlib import Path
import pandas as pd
import numpy as np
from mcmc_norm_learning.algorithm_1_v4 import to_tuple
from pickle_wrapper import unpickle

In [3]:
def write_log(output,path):
    with open(Path(path), 'w') as f:
        f.write(output)

## Loop over w_nc values
### Trial=1

In [4]:
exp_pattern="exp_nc5/w_nc={},trial={}"

In [5]:
!rm -r ../data_nc/exp_nc5/*

zsh:1: no matches found: ../data_nc/exp_nc5/*


In [6]:
w_nc_list=[round(0.05*i,2) for i in range(12)]
w_nc_list

[0.0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55]

In [7]:
w_nc_parts={
    f"part_{part}": w_nc_list[i*2:(i+1)*2]\
    for part,i in zip(["a","b","c","d","e","f"],range(int(len(w_nc_list)/2)+1))\
}

In [8]:
w_nc_parts

{'part_a': [0.0, 0.05],
 'part_b': [0.1, 0.15],
 'part_c': [0.2, 0.25],
 'part_d': [0.3, 0.35],
 'part_e': [0.4, 0.45],
 'part_f': [0.5, 0.55]}

In [9]:
import time
for trial in [1,2,3]:
    for part,w_nc_sub in w_nc_parts.items():
        for w_nc in tqdm(w_nc_sub,f"Loop Over diff w_nc: Part {part}"):
            exp_dir_name=exp_pattern.format(w_nc,trial)
            with io.capture_output() as exp_log:
                !python ../scripts/nc_experiments.py -exp $exp_dir_name -w_nc $w_nc  -n_threads 7
            write_log(exp_log.stdout,"../data_nc/{}/run.log".format(exp_dir_name))
            time.sleep(10)

## Check if lik values are as expected
### 1. True Norm log_posterior for diff w_nc

In [10]:
exp_pattern

'exp_nc5/w_nc={},trial={}'

In [11]:
from numpy import log
100*log(1/243)

-549.3061443340548

In [12]:
for trial in [1,2,3]:
    for w_nc in w_nc_list:
        exp_dir_name=exp_pattern.format(w_nc,trial)
        prior_out=!grep "For True Norm" ../data_nc/{exp_dir_name}/run.log
        post_out=!grep "lik_no_norm" ../data_nc/{exp_dir_name}/run.log
        print (f"For w_nc={w_nc}:")
        print (prior_out)
        print (post_out)
        print()

For w_nc=0.0:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
['lik_no_norm=-549.3061443340544,lik_true_norm=-522.2848671932203']

For w_nc=0.05:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
['lik_no_norm=-549.3061443340544,lik_true_norm=-519.092988336261']

For w_nc=0.1:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
['lik_no_norm=-549.3061443340544,lik_true_norm=-524.9618152263345']

For w_nc=0.15:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
['lik_no_norm=-549.3061443340544,lik_true_norm=-525.9182147369628']

For w_nc=0.2:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
['lik_no_norm=-549.3061443340544,lik_true_norm=-522.2426149850588']

For w_nc=0.25:
['For True Norm, prior=8.930612711476907e-07, log_prior=-13.928610645716274']
['lik_no_norm=-549.3061443340544,lik_true_norm=-521.831469788085']

For w_nc=0.3:
['For True Norm, pr

### 2. True Norm atleast mentioned in any chains, for diff w_nc

In [ ]:
# for w_nc in w_nc_list:
#     exp_dir_name=exp_pattern.format(w_nc)
#     out=!grep "True norm in some chain(s)" ../data_nc/{exp_dir_name}/chain_info.txt
#     print (f"For w_nc={w_nc}, True Norm found={out}")

### 3. Rank of True Norm as per log_post, iff it was found

In [ ]:
# with open("../params_nc.yaml", 'r') as fd:
#     params = yaml.safe_load(fd)
# true_norm=to_tuple(params['true_norm']['exp'])

In [ ]:
# for w_nc in w_nc_list:
#     exp_dir_name=exp_pattern.format(w_nc)
#     rank_df=pd.read_csv(f"../data_nc/{exp_dir_name}/ranked_posteriors.csv",index_col=False)
#     rank_post=rank_df.loc[rank_df.expression==str(true_norm)][["post_rank","log_posterior"]].values
#     if rank_post.shape[0]==1:
#         print (f"For w_nc={w_nc}, True Norm rank={rank_post[0][0]},wi posterior={rank_post[0][1]}")

#### ==> True Norm always has highest posterior, when its found,
confirm if it is in one chain, or multiple

### 4. In how many chains, max post crosses post(true_norm)

In [ ]:
# stacked=pd.DataFrame()
# for w_nc in w_nc_list:
#     exp_dir_name="exp_nc2/w_nc={}".format(w_nc)
#     result = pd.read_csv(f"../data_nc/{exp_dir_name}/chain_posteriors_nc.csv")
#     result["w_nc"] = w_nc
#     stacked=stacked.append(result)

In [ ]:
# grouped=stacked.groupby(["w_nc","chain_number"])[["log_posterior"]].agg(['min', 'max', 'mean', 'std'])

In [ ]:
# true_post={
#     "a":-25177,"b":-25782,\
#     "c":1,"d":-25773,\
#     "e":-25884,"f":1
# }

In [ ]:
# grouped["w_nc"]=[x[0] for x in grouped.index]
# grouped["key"]=np.where(grouped["w_nc"]==0.0,"a",\
#                         np.where(grouped["w_nc"]==0.05,"b",\
#                                 np.where(grouped["w_nc"]==0.1,"c",\
#                                         np.where(grouped["w_nc"]==0.15,"d",\
#                                                 np.where(grouped["w_nc"]==0.2,"e","f")\
#                                                 )\
#                                         )\
#                                 )\
#                         )

In [ ]:
# grouped.style.apply(lambda x: ['background: lightgreen' if x['log_posterior',  'max'] >=\
#                                true_post[x["key"].values.item()] else '' for i in x], 
#                axis=1)

### 5. If True Norm, features in Top Norms

In [ ]:
# top_norms=stacked.groupby(["w_nc","expression"]\
#                )[["chain_number","log_posterior"\
#                  ]].agg({"log_posterior":'mean',"chain_number":"count"}\
#                        ).sort_values(by=["chain_number"]\
#                                     ).groupby(["w_nc"\
#                                               ]).tail(20).sort_values(by=["w_nc","chain_number"],\
#                                                                      ascending=[True, False])

In [ ]:
# top_norms["grp_rnk"]=top_norms.groupby("w_nc")[["chain_number"]].rank(method='dense',ascending=False)

In [ ]:
# top_norms.loc[0.0].head(10).style.apply(lambda x: ['background: lightgreen' if x['log_posterior'] >=\
#                                true_post["a"] else '' for i in x], 
#                axis=1)

In [ ]:
# top_norms.loc[0.05].head(4).style.apply(lambda x: ['background: lightgreen' if x['log_posterior'] >=\
#                                true_post["b"] else '' for i in x], 
#                axis=1)

In [ ]:
# true_norm

#### Below top norms(1,2) equivalent norm to true norm for a very peculiar action seq:
1. posterior is lowered due to low prior (~complex condition)

In [ ]:
# top_norms.loc[0.1].head(4).style.apply(lambda x: ['background: lightgreen' if x['log_posterior'] >=\
#                                true_post["c"] else '' for i in x], 
#                axis=1)

In [ ]:
# top_norms.loc[0.15].head(4).style.apply(lambda x: ['background: lightgreen' if x['log_posterior'] >=\
#                                true_post["d"] else '' for i in x], 
#                axis=1)

In [ ]:
# top_norms.loc[0.2].head(10).style.apply(lambda x: ['background: lightgreen' if x['log_posterior'] >=\
#                                true_post["e"] else '' for i in x], 
#                axis=1)